## introduction

I am using my home PC, with Debian 12 "bookworm" (testing) and an NVIDIA GPU.

The following setup is **not recommended** unless you know what you're doing with Linux, and probably not even then.

I decided not to use conda, virtualenv, or docker. Instead, I installed the necessary Python modules under `/usr/local` with `pip`. This way I can use my AI tools directly from the command line and other scripts, like any other tools.

## allow my user to install packages to /usr/local


It's not safe to run `pip install -U` as root. It will merrily remove files from Debian-packaged Python modules under `/usr/lib`, and mess up the system. To avoid this, I changed permissions so that my regular user account can write to everything under `/usr/local` via the `staff` group. I do the same for `/opt` because I want to upgrade Rust from my own user ID also:

In [ ]:
sudo adduser $USER staff
sudo chgrp -R staff /usr/local /opt
sudo chmod -R g+w /usr/local /opt

adduser: The user `sam' is already a member of `staff'.


This is somewhat of a security concern, but if a bad guy gets a local shell it's pretty much game over anyway. To make it safer, I could have used a different non-root account.

In order to upgrade a pip package where the same package was also installed in `/usr/lib` by dpkg, I need to use `pip -I -U packagename`. The `-I` flag tells it to ignore installed packages, i.e. don't try to remove them. I only do that as needed for individual packages, when a normal upgrade fails.

This setup might upset Debian-packaged programs that depend on older versions of the Python libraries that I've upgraded. I haven't noticed any problems like that yet.

If you try to downgrade a package, and If multiple versions of a package are accidentally installed under /usr/local/lib/python3.10 at that same time, the sym

## enable switching the system python version

The default Python version for Debian "testing" at time of writing is Python 3.11, however we don't have a stable release of pytorch for Python 3.11 yet:

In [ ]:
python3.11 -m pip install torchvision

ERROR: Could not find a version that satisfies the requirement torchvision (from versions: 0.1.6, 0.1.7, 0.1.8, 0.1.9, 0.2.0, 0.2.1, 0.2.2, 0.2.2.post2, 0.2.2.post3)
ERROR: No matching distribution found for torchvision



: 1

So let's enable switching the system Python version back to 3.10, using `update-alternatives`.

First, note that we can undo this change later, if necessary, as follows:

In [ ]:
sudo update-alternatives --remove-all python3
sudo update-alternatives --remove-all pydoc3
sudo ln -sf python3.11 /usr/bin/python3
sudo ln -sf pydoc3.11 /usr/bin/pydoc3

We can set up alternatives for `python3`:

In [ ]:
sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.11 11
sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.10 10
sudo update-alternatives --set python3 /usr/bin/python3.10

update-alternatives: using /usr/bin/python3.11 to provide /usr/bin/python3 (python3) in auto mode
update-alternatives: using /usr/bin/python3.10 to provide /usr/bin/python3 (python3) in manual mode


and do the same for `pydoc3`:

In [ ]:
sudo update-alternatives --install /usr/bin/pydoc3 pydoc3 /usr/bin/pydoc3.11 11
sudo update-alternatives --install /usr/bin/pydoc3 pydoc3 /usr/bin/pydoc3.10 10
sudo update-alternatives --set pydoc3 /usr/bin/pydoc3.10

update-alternatives: using /usr/bin/pydoc3.11 to provide /usr/bin/pydoc3 (pydoc3) in auto mode
update-alternatives: using /usr/bin/pydoc3.10 to provide /usr/bin/pydoc3 (pydoc3) in manual mode


## install rust

Some Python modules now depend on Rust to build. I want to install Rust system-wide, in `/opt/rust`:

In [ ]:
cd /tmp
curl --proto '=https' --tlsv1.2 -sSf https://sh.rustup.rs > install_rust.sh
sudo RUSTUP_HOME=/opt/rust CARGO_HOME=/opt/rust sh ./install_rust.sh -y --no-modify-path

info: downloading installer
info: (Bprofile set to 'default'
info: (Bdefault host triple is x86_64-unknown-linux-gnu
info: (Bsyncing channel updates for 'stable-x86_64-unknown-linux-gnu'
info: (Bdefault toolchain set to 'stable-x86_64-unknown-linux-gnu'

  stable-x86_64-unknown-linux-gnu unchanged(B - rustc 1.67.1 (d5a82bbd2 2023-02-07)


Rust is installed now. Great!
(B
To get started you need Cargo's bin directory (/opt/rust/bin) in your PATH(B
environment variable. This has not been done automatically.

To configure your current shell, run:
source "/opt/rust/env"


In [ ]:
. /opt/rust/env
rustup default stable

info: (Busing existing install for 'stable-x86_64-unknown-linux-gnu'
info: (Bdefault toolchain set to 'stable-x86_64-unknown-linux-gnu'

  stable-x86_64-unknown-linux-gnu unchanged(B - rustc 1.67.1 (d5a82bbd2 2023-02-07)



## install packages for fastai and jupyter

In [ ]:
pip install -U -qq fastbook jupyter jupyterlab tensorflow

Extra packages for bash and C kernels:

In [ ]:
pip install -U -qq nbdev bash_kernel jupyter-c-kernel

In [ ]:
python -m bash_kernel.install

Installing IPython kernel spec


In [ ]:
install_c_kernel

Installing IPython kernel spec
/usr/local/bin/install_c_kernel:32: DeprecationWarning: replace is ignored. Installing a kernelspec always replaces an existing installation
  KernelSpecManager().install_kernel_spec(td, 'c', user=user, replace=True, prefix=prefix)
